In [ ]:
"""
Dynamic life cycle assessment main code

Authors: 
- Augustin Danneaux
- Estelle Schurer
- Alperen Yayla
"""

# Importing libraries

from scipy.stats import norm
import numpy as np
import pandas as pd
import os
import time
from tqdm import tqdm

# Importing modules

import core_modules.core_utilities as cu
import core_modules.core_climate as cc
import core_modules.core_decay_plot_functions as cp

# Analysis type

ind_GWP = False
ind_AGTP = not ind_GWP # Set so code does not calculate GWP and AGTP at the same time which would take a long time
ind_plot = True # True if only ind_AGTP is True
ind_regional = True

# Importing Data

input_path = os.path.join('..','generated_data')
output_path = os.path.join('..','output/results')

K119       = pd.read_csv(os.path.join(input_path,'K119.csv')).values
#K126       = pd.read_csv(os.path.join(input_path,'K126.csv')).values
K245       = pd.read_csv(os.path.join(input_path,'K245.csv')).values
K370       = pd.read_csv(os.path.join(input_path,'K370.csv')).values
#K585       = pd.read_csv(os.path.join(input_path,'K585.csv')).values

Fd_RCA     = pd.read_csv(os.path.join(input_path,'Fd_RCA_BAU.csv')).values
Fd_Unb     = pd.read_csv(os.path.join(input_path,'Fd_Unbound_BAU.csv')).values
Fd_Lan     = [x[0] for x in pd.read_csv(os.path.join(input_path,'Fd_Landfill_BAU.csv')).values]
Fd_RCA_LC3 = pd.read_csv(os.path.join(input_path,'Fd_RCA_LC3.csv')).values
Fd_Unb_LC3 = pd.read_csv(os.path.join(input_path,'Fd_Unbound_LC3.csv')).values
Fd_Lan_LC3 = [x[0] for x in pd.read_csv(os.path.join(input_path,'Fd_Landfill_LC3.csv')).values]

regional_file = '../generated_data/SSPs_buildings.xlsx'
ssp1_ub = pd.read_excel(regional_file, 'buildings-ssp1')
ssp2_ub = pd.read_excel(regional_file, 'buildings-ssp2')
ssp3_ub = pd.read_excel(regional_file, 'buildings-ssp3')

if ind_GWP:
    LCI_data  = pd.read_csv(os.path.join(input_path,'Scenario_LCI.csv'),index_col=0)
elif ind_AGTP:
    LCI_data  = pd.read_csv(os.path.join(input_path,'Input_AGTP.csv'),index_col=0)
else:
    print('Error: No analysis type selected')

Fd_EOL = [Fd_RCA[x][0]*0.23+Fd_Unb[x][0]*0.77 for x in range(0,len(Fd_RCA))]
Fd_EOL_LC3 = [Fd_RCA_LC3[x][0]*0.23+Fd_Unb_LC3[x][0]*0.77 for x in range(0,len(Fd_RCA_LC3))]

Fd_EOL3 = [Fd_Lan[x]*0.213+Fd_EOL[x]*0.787 for x in range(0,len(Fd_RCA))]
Fd_EOL3_LC3 = [Fd_Lan_LC3[x]*0.213+Fd_EOL_LC3[x]*0.787 for x in range(0,len(Fd_RCA))]

# Scenario dependent Parameters
# Amount of raw pine

Cemtype =[0         ,0          ,0          ,1          ,1          ,1]
LFD     =[Fd_EOL    ,Fd_Lan     ,Fd_EOL3    ,Fd_EOL_LC3 ,Fd_Lan_LC3 ,Fd_EOL3_LC3 ]

# Constant parameters

MCar2  = [130.6/0.8,71.82088/0.8]
ConcreteVolume = 1169.456 #[m3]
kSCM = [1,1.7]

TOD = 500           #[years]
r = 100            #[years]
s = 100            #[years]
Life = 100         #[years]

n = TOD*200+1
t_TOD = np.linspace(0,TOD,n) 
dt = t_TOD[1]-t_TOD[0]

rf_CO2, rf_CH4, rf_N2O=cc.define_rf()
Rt = cc.define_rt(t_TOD)

# Convolution functions

yCO2_TOD = cc.decay_function('CO2',t_TOD)

Empty_pulse = 0*t_TOD

SSPn = ['1-19','2-45','3-70']
name_array = []

# Building numbers

regions=ssp2_ub['Region']
regioncodes=ssp2_ub['DLCA code']
region_inputs = [{'region': region, 'code': code} for region, code in zip(regions, regioncodes)]

building_types = list(LCI_data.columns)

# Calculate the total number of iterations for the progress bar

total_iterations = len(region_inputs) * len(building_types)

# Initialize the single progress bar

pbar = tqdm(total=total_iterations,
            #miniters=None,
            dynamic_ncols=True,
            bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt} | {elapsed}<{remaining}",
            position=0)

for information in region_inputs:
    
    # Data information and labelling
    
    regionname = information['region']
    regioncode = information['code']
    region = regioncode
    
    # Update the progress bar description
    
    pbar.set_description(f'Starting DLCA for {regioncode}, {regionname}')
    
    ssp_dict = {
    'ssp1': ssp1_ub,
    'ssp2': ssp2_ub,
    'ssp3': ssp3_ub}
    
    region_buildings = {}
    
    # Looping through buildings

    GWP_C_net_array = []
    GWP_C_nonbiogenic_array = []
    GWP_C_biogenic_array = []
    GWP_M_array = []
    GWP_N_array = []
    GWP_L_carb_array = []
    GWP_EOL_carb_array = []
    GWP_MN_array = []
    GWP_Net_array = []
    AGTP_results = pd.DataFrame()
    
    for ssp_name, ssp_ub in ssp_dict.items():
    
        region_buildings_ssp = cu.get_building_numbers(ssp_ub, regioncode)
        region_buildings_ssp = region_buildings_ssp[regioncode].to_numpy()
        region_buildings[f'region_buildings_{ssp_name}'] = region_buildings_ssp

    for building_type in building_types:

        # Update dynamic description with the current region and building type
        
        pbar.set_description(f'Processing region: {regionname} | Building type: {building_type}')
        
        # Start timer
        
        start_time = time.time()

        Dynamic = bool(LCI_data.loc['Dynamic',building_type])
        SSP = int(LCI_data[building_type]['SSP'])
        THI = float(LCI_data[building_type]['THI'])
        
        # Get data
        
        incineration_share = float(LCI_data[building_type]['incineration_share']) #[-]

        pine_mass= float(LCI_data[building_type]['C_Pine_Mass']) #[kg] Version 1 (CLT building)

        building_CO2 = pine_mass*0.5*3.67 #kilograms of CO2 in a building
        degradable_carbon = pine_mass*0.5*0.105 #kilograms of C (10.5% of wood subject to decay)


        Data_C  =cu.get_LCI_data(LCI_data,building_type,'C')
        Data_C['Biopulse'] = pine_mass*0.5*3.67 #[kg] C & CO2 conversion

        Data_M  =cu.get_LCI_data(LCI_data,building_type,'M')
        Data_N  =cu.get_LCI_data(LCI_data,building_type,'N')

        ConcreteVolume= float(LCI_data[building_type]['ConcreteVolume'])   #[m3]
        Area_in= float(LCI_data[building_type]['Area_in'])                 #[m2]
        Area_out= float(LCI_data[building_type]['Area_out'])              #[m2]

        Cemtype= int(LCI_data[building_type]['Cemtype'])           #[-]
        LFD_index= int(LCI_data[building_type]['LFD_index'] )      #[-]

        landfill_Ashare = float(LCI_data.loc['landfill_Ashare',building_type])         #[-]
        landfill_Bshare = float(LCI_data.loc['landfill_Bshare',building_type])         #[-]
        landfill_Cshare = float(LCI_data.loc['landfill_Cshare',building_type])         #[-]

        recycling_share = float(LCI_data[building_type]['recycling_share'] )        #[-]
        regrowth_share = float(LCI_data[building_type]['regrowth_share'] ) 
        
        TexposureEOL = float(LCI_data[building_type]['TexposureEOL'] )       #[years]

        # Loop dependent parameters
        
        MCar  = MCar2[Cemtype]
        k_out = 2.25*1e-3 *kSCM[Cemtype]        #[m/year1/2]
        k_in = 6.3*1e-3   *kSCM[Cemtype]        #[m/year1/2]
        K = k_out*1e3
        Mmax = MCar*ConcreteVolume

        Fd = [x for x in LFD[LFD_index]]

        # Loop dependent Time parameters
        
        t = t_TOD[t_TOD<=THI]
        Rtt = Rt[t_TOD<=THI]

        # Convolution
    
        denom = [yCO2_TOD[0]]
        for x in range(1,len(yCO2_TOD)):
            denom.append(denom[x-1]+yCO2_TOD[x])
        denom = np.array(denom)

        denom0   = float(denom[t_TOD==0+THI][0])

        denomz = {}
        denomz['SOL'] = denom0
        denomz['EOL'] = float(denom[t_TOD==Life+THI][0])
        denomz['incineration_pulse'] = float(denom[t_TOD==Life+THI][0])
        denomz['CRE'] = float(denom[int((Life+TexposureEOL)/dt)])
        denomz['EOL_bio_emissions'] = 1 # Denominator is included when calculating fv
        denomz['EOL_bio_credit']    = 1 # Denominator is included when calculating f
        denom    = denom[t_TOD>=THI]

        yCO2 = cc.decay_function('CO2',t)
        yCH4 = cc.decay_function('CH4',t)
        yN2O = cc.decay_function('N2O',t)

        # Initiate emissions dictionaries
        
        Emissions_C = cu.set_emissions(Data_C)
        Emissions_M = cu.set_emissions(Data_M)
        Emissions_N = cu.set_emissions(Data_N)
        Emissions_C['incineration_pulse'] = Data_C['Biopulse'] * incineration_share

        # Place emissions in the pulse dictionaries
        
        Pulse_C = cu.place_emissions_in_pulse(Emissions_C, t_TOD, Life, Dynamic)
        Pulse_M = cu.place_emissions_in_pulse(Emissions_M, t_TOD, Life, Dynamic)
        Pulse_N = cu.place_emissions_in_pulse(Emissions_N, t_TOD, Life, Dynamic)

        # Carbonation

        # Calculating total carbonation potentials
        
        M = (Area_in*k_in+Area_out*k_out)*np.sqrt(t_TOD)*MCar
        M[t_TOD>=Life]=M[t_TOD<Life][-1]
        ML= (Area_in*k_in+Area_out*k_out)*np.sqrt(Life)*MCar
        McarEOL = (Mmax-ML)*np.array(Fd)
        McarEOL[t_TOD[:-1]>=Life+TexposureEOL] = McarEOL[t_TOD[:-1]<Life+TexposureEOL][-1]

        M = [M[x]+McarEOL[x] for x in range(0,len(M)-1)]
        M = np.array(M)
        M[M>Mmax] = Mmax
        dMdt = []
        dMdtSSP = []
        dMdt_for_GWP = []

        # Assessing effect of future CO2 concentration on carbonation
        
        K = [K119,K245,K370][SSP]

        for j in range(0,len(denom)):
            dMdt.append((M[j+1]-M[j]))
            dMdt_for_GWP.append(-dMdt[-1] / denom[j])
            dMdtSSP.append(dMdt[j]*K[j])

        MSSP = [0]
        for j in range(0,len(denom)):
            MSSP.append(float((MSSP[-1] + dMdtSSP[j]).item()))

        MSSP = np.array(MSSP)

        MSSP[MSSP>Mmax]=Mmax

        dMdtSSP = []
        dMdtSSP_for_GWP = []
            
        for j in range(0,len(denom)):
            dMdtSSP.append((MSSP[j+1]-MSSP[j]))
            dMdtSSP_for_GWP.append(-dMdtSSP[-1]/denom[j])

        dMdtSSP_for_GWP_L   = [0 if t_TOD[x]>Life-2*dt else dMdtSSP_for_GWP[x] for x in range(len(dMdtSSP_for_GWP)) ]
        dMdtSSP_for_GWP_EOL = [0 if t_TOD[x]<=Life-2*dt else dMdtSSP_for_GWP[x] for x in range(len(dMdtSSP_for_GWP)) ]

        # Biomass Sink = Gaussian
        
        mu, sigma = r/2, r/4
        g = - dt*Data_C['Biopulse']*norm.pdf(t_TOD,mu,sigma)
        g_credit = dt*Data_C['Biopulse']*norm.pdf(t_TOD-Life,mu,sigma)*recycling_share
        rg_credit = dt*Data_C['Biopulse']*norm.pdf(t_TOD-Life,mu,sigma)*regrowth_share*-1
        g_credit[t_TOD<s]=0
        rg_credit[t_TOD<s]=0

        g_for_GWP = np.array(g[:len(denom)])/np.array(denom)
        g_credit_for_GWP  = np.array(g_credit[:len(denom)])/np.array(denom)
        rg_credit_for_GWP  = np.array(rg_credit[:len(denom)])/np.array(denom)
        
        # Lanfill decay
        
        EoL_decay_f = cc.EOL_decay_functions(t_TOD,s,dt)

        #Methane heat + electricity co-production (ecoinvent): 
        # 0.249 m3 of natural gas= 3.6 MJ elec + 1.63 MJ heat
        
        CH4_burning = 0.249*0.657e-3/(3.6+1.63) # tonnes of CH4/MJ of energy

        Data_C['EOL_bio_credit']=-6.30E-05 #tonnes of CO2/MJ of energy (69/31 split elec/heat)
        Data_M['EOL_bio_credit']=-1.37E-06 #tonnes of CH4/MJ of energy
        Data_N['EOL_bio_credit']=-4.27E-09 #tonnes of N2O/MJ of energy

        Emissions_C['EOL_bio_emissions'] = degradable_carbon*(3.67) * (landfill_Ashare*10/100+ landfill_Bshare*99.7/100+ landfill_Cshare*99.995/100)
        Emissions_M['EOL_bio_emissions'] = degradable_carbon*(1.33) * (landfill_Ashare*90/100+ landfill_Bshare*0.03/100+ landfill_Cshare*0.005/100)
        Emissions_N['EOL_bio_emissions'] = 0

        Emissions_C['EOL_bio_credit'] = degradable_carbon*(1.33)/CH4_burning*Data_C['EOL_bio_credit']*landfill_Cshare
        Emissions_M['EOL_bio_credit'] = degradable_carbon*(1.33)/CH4_burning*Data_M['EOL_bio_credit']*landfill_Cshare
        Emissions_N['EOL_bio_credit'] = degradable_carbon*(1.33)/CH4_burning*Data_N['EOL_bio_credit']*landfill_Cshare

        for pulse in ['EOL_bio_emissions','EOL_bio_credit']:
            Pulse_C[pulse]=Emissions_C[pulse]*EoL_decay_f
            Pulse_M[pulse]=Emissions_M[pulse]*EoL_decay_f
            Pulse_N[pulse]=Emissions_N[pulse]*EoL_decay_f

        # Emission Decay

        f_C =cu.create_basic_convolution(yCO2,Pulse_C)
        f_M =cu.create_basic_convolution(yCH4,Pulse_M)
        f_N =cu.create_basic_convolution(yN2O,Pulse_N)

        if Dynamic:
            
            # Convoluting the carbonation sink
            
            f_C_Carb_L = np.convolve(yCO2,dMdtSSP_for_GWP_L)
            
            f_C_Carb_EOL = np.convolve(yCO2,dMdtSSP_for_GWP_EOL)
            f_C_Carb_notdivided = np.convolve(yCO2,[-x for x in dMdtSSP])
            
            # Convoluting forestry sink
            
            f_C_G = np.convolve(yCO2,g_for_GWP)
            f_C_G_credit = np.convolve(yCO2,g_credit_for_GWP)
            f_C_RG_credit = np.convolve(yCO2,rg_credit_for_GWP)

            f_C_G_notdivided = np.convolve(yCO2,g)
            f_C_G_credit_notdivided = np.convolve(yCO2,g_credit)
            f_C_RG_credit_notdivided = np.convolve(yCO2,rg_credit)

            # Convoluting EoL decay
            
            f_C = cu.EOL_bio_convolutions(f_C,yCO2,Pulse_C,denom)
            f_M = cu.EOL_bio_convolutions(f_M,yCH4,Pulse_M,denom)
            f_N = cu.EOL_bio_convolutions(f_N,yN2O,Pulse_N,denom)

            if ind_regional:

                ssp_key = building_type.split('_')[-1]
                region_key = f'region_buildings_{ssp_key.lower()}'
                ssp_buildings = region_buildings[region_key]

                building_CO2_storage = 1 * np.concatenate((np.full(20000, building_CO2), np.zeros(40000)))

                # Define a list of variable names to update directly
                
                variables = [
                    'f_C_Carb_L', 'f_C_Carb_EOL', 'f_C_Carb_notdivided',
                    'f_C_G', 'f_C_G_credit', 'f_C_RG_credit', 'f_C_G_notdivided', 'f_C_G_credit_notdivided', 'f_C_RG_credit_notdivided', 'building_CO2_storage'
                ]

                # Update the variables using a loop
                
                for var in variables:
                    globals()[var] = cu.get_region_scale(ssp_buildings, globals()[var]) 

                # Define a list of dictionary pulses to update within each dictionary
                
                pulses = [
                    'SOL', 'EOL', 'CRE', 'incineration_pulse', 'EOL_bio_emissions',
                    'EOL_bio_credit', 'EOL_bio_emissions_notdivided', 'EOL_bio_credit_notdivided'
                ]

                # Update the dictionaries using nested loops
                
                for pulse in pulses:
                    f_C[pulse] = cu.get_region_scale(ssp_buildings, f_C[pulse])
                    f_M[pulse] = cu.get_region_scale(ssp_buildings, f_M[pulse])
                    f_N[pulse] = cu.get_region_scale(ssp_buildings, f_N[pulse])

            # GWP composition
            
            GWP_Carb_L = np.sum(f_C_Carb_L)
            GWP_Carb_EOL = np.sum(f_C_Carb_EOL)
            GWP_G   = np.sum(f_C_G)+np.sum(f_C_G_credit)+np.sum(f_C_RG_credit)
            
            GWP_C = cc.add_dynamic_GWP(f_C,denomz,rf_CO2)
            GWP_M = cc.add_dynamic_GWP(f_M,denomz,rf_CH4)
            GWP_N = cc.add_dynamic_GWP(f_N,denomz,rf_N2O)
            
            # Total GWP
            
            GWP = GWP_Carb_L + GWP_Carb_EOL + GWP_C + GWP_M +  GWP_N  + GWP_G 

            f_C = cu.calculate_f_net(f_C,t_TOD)
            f_M = cu.calculate_f_net(f_M,t_TOD)
            f_N = cu.calculate_f_net(f_N,t_TOD)

            f_C['Net'] += f_C_Carb_notdivided[:len(t_TOD)] + f_C_G_notdivided[:len(t_TOD)] + f_C_G_credit_notdivided[:len(t_TOD)] + f_C_RG_credit_notdivided[:len(t_TOD)]

        else:
            
            # Convoluting the carbonation sink
            
            P_C_carb_L = np.zeros(len(t_TOD))
            P_C_carb_EOL = np.zeros(len(t_TOD))
            P_C_carb_L[t_TOD==0] = -MSSP[np.where(t_TOD==Life-dt)]
            P_C_carb_EOL[t_TOD==0] = -MSSP[-1]+MSSP[np.where(t_TOD==Life-dt)]
                
            f_C_Carb_L = 	np.convolve(yCO2,P_C_carb_L)
            f_C_Carb_EOL = 	np.convolve(yCO2,P_C_carb_EOL)

            # Convoluting forestry sink
            
            P_C_G = np.zeros(len(t_TOD))
            P_C_G[t_TOD==0] = np.sum(g)
            P_C_G_credit = np.zeros(len(t_TOD))
            P_C_RG_credit = np.zeros(len(t_TOD))
            P_C_G_credit[t_TOD==0] = np.sum(g_credit)
            P_C_RG_credit[t_TOD==0] = np.sum(rg_credit)

            f_C_G = np.convolve(yCO2,P_C_G)
            f_C_G_credit = np.convolve(yCO2,P_C_G_credit)
            f_C_RG_credit = np.convolve(yCO2,P_C_RG_credit)

            f_C_G_notdivided = np.convolve(yCO2,P_C_G)
            f_C_G_credit_notdivided = np.convolve(yCO2,P_C_G_credit)
            f_C_RG_credit_notdivided = np.convolve(yCO2,P_C_RG_credit)

            # Convoluting EoL decay
            
            for pulse in['EOL_bio_emissions','EOL_bio_credit']:
                P_C = np.sum(Pulse_C[pulse])
                P_M = np.sum(Pulse_M[pulse])
                P_N = np.sum(Pulse_N[pulse])

                Pulse_C[pulse] = np.zeros(len(t_TOD))
                Pulse_M[pulse] = np.zeros(len(t_TOD))
                Pulse_N[pulse] = np.zeros(len(t_TOD))

                Pulse_C[pulse][t_TOD==0] = P_C
                Pulse_M[pulse][t_TOD==0] = P_M
                Pulse_N[pulse][t_TOD==0] = P_N
                
                f_C[pulse] = 	np.convolve(yCO2,Pulse_C[pulse])
                f_M[pulse] = 	np.convolve(yCH4,Pulse_M[pulse])
                f_N[pulse] = 	np.convolve(yN2O,Pulse_N[pulse])

            if ind_regional:

                ssp_key = building_type.split('_')[-1]
                region_key = f'region_buildings_{ssp_key.lower()}'
                ssp_buildings = region_buildings[region_key]

                building_CO2_storage = 1 * np.concatenate((np.full(20000, building_CO2), np.zeros(40000)))

                # Define a list of variable names to update directly
                
                variables = [
                    'f_C_Carb_L', 'f_C_Carb_EOL',
                    'f_C_G', 'f_C_G_credit', 'f_C_RG_credit', 'f_C_G_notdivided', 'f_C_G_credit_notdivided', 'f_C_RG_credit_notdivided', 'building_CO2_storage'
                ]

                # Update the variables using a loop
                
                for var in variables:
                    globals()[var] = cu.get_region_scale(ssp_buildings, globals()[var])  

                # Define a list of dictionary pulses to update within each dictionary
                
                pulses = [
                    'SOL', 'EOL', 'CRE', 'incineration_pulse', 'EOL_bio_emissions',
                    'EOL_bio_credit'
                ]

                # Update the dictionaries using nested loops
                
                for pulse in pulses:
                    f_C[pulse] = cu.get_region_scale(ssp_buildings, f_C[pulse])
                    f_M[pulse] = cu.get_region_scale(ssp_buildings, f_M[pulse])
                    f_N[pulse] = cu.get_region_scale(ssp_buildings, f_N[pulse])

            # GWP composition
                    
            GWP_Carb_L = np.sum(f_C_Carb_L)/denom0
            GWP_Carb_EOL = np.sum(f_C_Carb_EOL)/denom0
            GWP_G   = np.sum(f_C_G)/denom0+np.sum(f_C_G_credit)/denom0+np.sum(f_C_RG_credit)/denom0

            GWP_C =0
            GWP_M =0
            GWP_N =0

            f_C['Net'] = 0
            f_M['Net'] = 0
            f_N['Net'] = 0

            for pulse in ['SOL',  'EOL', 'CRE', 'EOL_bio_emissions','EOL_bio_credit']:
                f_C['Net'] += f_C[pulse][:len(t_TOD)]
                f_M['Net'] += f_M[pulse][:len(t_TOD)]
                f_N['Net'] += f_N[pulse][:len(t_TOD)]

                GWP_C += np.sum(f_C[pulse])/denom0
                GWP_M += np.sum(f_M[pulse])*rf_CH4/rf_CO2/denom0
                GWP_N += np.sum(f_N[pulse])*rf_N2O/rf_CO2/denom0

            f_C['Net'] += f_C_Carb_L[:len(t_TOD)] + f_C_Carb_EOL[:len(t_TOD)] + f_C_G_notdivided[:len(t_TOD)] + f_C_G_credit_notdivided[:len(t_TOD)] + f_C_RG_credit_notdivided[:len(t_TOD)]

            GWP = GWP_Carb_L + GWP_Carb_EOL + GWP_C + GWP_M +  GWP_N + GWP_G

        if ind_AGTP:
            RF_net = f_C['Net']*rf_CO2 + f_M['Net']*rf_CH4 + f_N['Net']*rf_N2O
            AGTP = np.convolve(Rt,RF_net)
            AGTP_results[building_type] = AGTP[0:len(t_TOD[t_TOD<300])]*dt

        if ind_GWP:
            GWP_C_net_array.append(GWP_C+ GWP_G + GWP_Carb_L + GWP_Carb_EOL)
            GWP_C_nonbiogenic_array.append(GWP_C)
            GWP_C_biogenic_array.append(GWP_G)
            GWP_L_carb_array.append(GWP_Carb_L)
            GWP_EOL_carb_array.append(GWP_Carb_EOL)
            GWP_M_array.append(GWP_M)
            GWP_N_array.append(GWP_N)
            GWP_MN_array.append(GWP_M +  GWP_N)
            GWP_Net_array.append(GWP)
        
        # Plotting Carbon decay
            
        if ind_plot:
            cp.plot_carbon_decay(building_type,f_C,f_C_Carb_notdivided,f_C_G_notdivided,f_C_G_credit_notdivided, f_C_RG_credit_notdivided, building_CO2_storage, t_TOD)
            cp.plot_methane_decay(building_type,f_M,t_TOD)

        pbar.update(1) # Update progress bar after each iteration

    # Output

    directory = output_path
    if not os.path.exists(directory):
        os.makedirs(directory) 

    if ind_GWP:
        Results = pd.DataFrame({'CO2_net':GWP_C_net_array,
                                'CO2_nonbio':GWP_C_nonbiogenic_array,
                                'CO2_bio':GWP_C_biogenic_array,
                                'Life_carbonation':GWP_L_carb_array,
                                'EOL_carbonation':GWP_EOL_carb_array,                          
                                'CH4':GWP_M_array,
                                'N2O':GWP_N_array,
                                'MN':GWP_MN_array,
                                'Net':GWP_Net_array},
                                index=building_types)

        Low_Error = []
        High_Error = []

        RCbuildings = np.array([[[[x+'_'+v+'_'+y+'_'+z for x in ['BAU','LC3'] for y in ['Dynamic','Static']] for z in ['20','100','200']]] for v in ['C1','C2','C3']]).flatten()
        ESTbuildings = np.array([[[[x+'_'+v+'_'+y+'_'+z for x in ['EST',] for y in ['Dynamic','Static']] for z in ['20','100','200']]] for v in ['T1','T2','T3', 'T4', 'T5', 'T6', 'T7', 'T8']]).flatten()
        Allbuildings = np.concatenate((RCbuildings, ESTbuildings))

        for building_type in Allbuildings:
            SSP1 = building_type + '_SSP1'
            SSP2 = building_type + '_SSP2'
            SSP3 = building_type + '_SSP3'

            low_error = Results['Net'][SSP2] - Results['Net'][SSP3]
            high_error = -Results['Net'][SSP2] + Results['Net'][SSP1]

            # Round to zero if negligible error
            
            low_error = 0 if abs(low_error) < 0.05 else low_error
            high_error = 0 if abs(high_error) < 0.05 else high_error

            Low_Error.append(low_error)
            High_Error.append(high_error)

        Errors = pd.DataFrame({'Low_Error':Low_Error,
                                'High_Error':High_Error},
                                index=[f"{bt}_SSP2" for bt in Allbuildings])

        Results=Results.merge(Errors, how='left', left_index=True, right_index=True)
        Results = Results.fillna(0)

        Results.to_csv(os.path.join(output_path,'GWP_results_'+regioncode+'.csv'))

    if ind_AGTP:
        AGTP_results['T']=t_TOD[t_TOD<300]
        AGTP_results.to_csv(os.path.join(output_path,'AGTP_results_'+regioncode+'.csv'))
        
    # Indicate completion of the region processing in the description
    
    pbar.set_description(f'DLCA is completed for {regioncode}, {regionname}')

pbar.close() # Close the progress bar   